In [1]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import glob
import os
import matplotlib.pyplot as plt

In [2]:
train_path = max(glob.glob('./data/train/*.csv'), key=os.path.getctime) 
test_path = max(glob.glob('./data/test/*.csv'), key=os.path.getctime) 
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print(train_df.shape)
print(test_df.shape)

(24756, 592)
(4369, 592)


In [26]:
print(train_df.columns[:25])
print(test_df.columns[:25])
# test_df.columns == train_df.columns

Index(['id', 'name', 'description', 'description_story', 'description_risks',
       'rewards', 'reward_tiers', 'min_reward', 'max_reward', 'category',
       'goal', 'deadline', 'location', 'state', 'staff_pick', 'video',
       'launched_at', 'created_at', 'has_video', 'rewards_processed',
       'description_processed', 'description_story_processed',
       'description_risks_processed', 'rewards_0', 'rewards_1'],
      dtype='object')
Index(['id', 'name', 'description', 'description_story', 'description_risks',
       'rewards', 'reward_tiers', 'min_reward', 'max_reward', 'category',
       'goal', 'deadline', 'location', 'state', 'staff_pick', 'video',
       'launched_at', 'created_at', 'rewards_processed',
       'description_processed', 'description_story_processed',
       'description_risks_processed', 'has_video', 'rewards_0', 'rewards_1'],
      dtype='object')


In [9]:
features_to_drop = ['rewards', 'deadline', 'launched_at', 'rewards_processed', 'created_at',
'description_processed', 'description_story_processed','description_risks_processed',
'id', 'name', 'description', 'description_story', 'description_risks', 'video', 'state',
'location', 'category']

#features that are dependent on time and the final outcome

X_train, y_train = train_df.drop(features_to_drop, axis=1), train_df['state']
X_test, y_test = test_df.drop(features_to_drop, axis=1), test_df['state']


In [10]:
# Combine train and test data set tgt

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [11]:
# Check if dataset is balanced
print(train_df[train_df.state == 1].shape)
print(train_df[train_df.state == 0].shape)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(17482, 592)
(7274, 592)
(24756, 575)
(24756,)
(4369, 575)
(4369,)


In [12]:
# Make sure all data are of integer of float type
X_train.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [18]:
model_xgboost1 = xgb.XGBClassifier(learning_rate=0.1,
                                      max_depth=5,
                                      n_estimators=5000,
                                      subsample=0.5,
                                      colsample_bytree=0.5,
                                      eval_metric='auc',
                                      verbosity=1)

model_xgboost1.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=[(X_test, y_test)],
                  verbose=True)

c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


ValueError: feature_names mismatch: ['reward_tiers', 'min_reward', 'max_reward', 'goal', 'staff_pick', 'has_video', 'rewards_0', 'rewards_1', 'rewards_2', 'rewards_3', 'rewards_4', 'rewards_5', 'rewards_6', 'rewards_7', 'rewards_8', 'rewards_9', 'rewards_10', 'rewards_11', 'rewards_12', 'rewards_13', 'rewards_14', 'rewards_15', 'rewards_16', 'rewards_17', 'rewards_18', 'rewards_19', 'rewards_20', 'rewards_21', 'rewards_22', 'rewards_23', 'rewards_24', 'rewards_25', 'rewards_26', 'rewards_27', 'rewards_28', 'rewards_29', 'rewards_30', 'rewards_31', 'rewards_32', 'rewards_33', 'rewards_34', 'rewards_35', 'rewards_36', 'rewards_37', 'rewards_38', 'rewards_39', 'rewards_40', 'rewards_41', 'rewards_42', 'rewards_43', 'rewards_44', 'rewards_45', 'rewards_46', 'rewards_47', 'rewards_48', 'rewards_49', 'rewards_50', 'rewards_51', 'rewards_52', 'rewards_53', 'rewards_54', 'rewards_55', 'rewards_56', 'rewards_57', 'rewards_58', 'rewards_59', 'rewards_60', 'rewards_61', 'rewards_62', 'rewards_63'

In [20]:
model_xgboost2 = xgb.XGBClassifier(learning_rate=0.1, eval_metric='auc')

model_xgboost2.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=[(X_test, y_test)],
                  verbose=True)

ValueError: feature_names mismatch: ['reward_tiers', 'min_reward', 'max_reward', 'goal', 'staff_pick', 'has_video', 'rewards_0', 'rewards_1', 'rewards_2', 'rewards_3', 'rewards_4', 'rewards_5', 'rewards_6', 'rewards_7', 'rewards_8', 'rewards_9', 'rewards_10', 'rewards_11', 'rewards_12', 'rewards_13', 'rewards_14', 'rewards_15', 'rewards_16', 'rewards_17', 'rewards_18', 'rewards_19', 'rewards_20', 'rewards_21', 'rewards_22', 'rewards_23', 'rewards_24', 'rewards_25', 'rewards_26', 'rewards_27', 'rewards_28', 'rewards_29', 'rewards_30', 'rewards_31', 'rewards_32', 'rewards_33', 'rewards_34', 'rewards_35', 'rewards_36', 'rewards_37', 'rewards_38', 'rewards_39', 'rewards_40', 'rewards_41', 'rewards_42', 'rewards_43', 'rewards_44', 'rewards_45', 'rewards_46', 'rewards_47', 'rewards_48', 'rewards_49', 'rewards_50', 'rewards_51', 'rewards_52', 'rewards_53', 'rewards_54', 'rewards_55', 'rewards_56', 'rewards_57', 'rewards_58', 'rewards_59', 'rewards_60', 'rewards_61', 'rewards_62', 'rewards_63'

**Evaluate Model Performance**

In [50]:
y_train_pred = model_xgboost1.predict_proba(X_train)[:,1]
y_test_pred = model_xgboost1.predict_proba(X_test)[:,1] # Slicing to obtain prob of observation being 1

print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

AUC Train: 0.9879
AUC Valid: 0.9225


In [51]:
y_train_pred = model_xgboost2.predict_proba(X_train)[:,1]
y_test_pred = model_xgboost2.predict_proba(X_test)[:,1] # Slicing to obtain prob of observation being 1

print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

AUC Train: 0.9997
AUC Valid: 0.9293


**Hyperparameter Tuning**

We will use GridSearchCV for hyperparameter tuning.


In [52]:
learning_rate_list = [0.02, 0.05, 0.1]
max_depth_list = [2, 3, 5]
n_estimators_list = [1000, 2000, 3000]

params_dict = {"learning_rate": learning_rate_list,
               "max_depth": max_depth_list,
               "n_estimators": n_estimators_list}

num_combinations = 1
for v in params_dict.values(): num_combinations *= len(v) 

print(num_combinations)
params_dict

27


{'learning_rate': [0.02, 0.05, 0.1],
 'max_depth': [2, 3, 5],
 'n_estimators': [1000, 2000, 3000]}

In [54]:
def auc_score(model, X, y): 
    return roc_auc_score(y, model.predict_proba(X)[:,1])

model_xgboost_hp = GridSearchCV(estimator=xgb.XGBClassifier(subsample=0.5,
                                                                colsample_bytree=0.25,
                                                                eval_metric='auc',
                                                                use_label_encoder=False),
                                param_grid=params_dict,
                                cv=2,
                                scoring=auc_score,
                                return_train_score=True,
                                verbose=4)

model_xgboost_hp.fit(X, y)

Fitting 2 folds for each of 27 candidates, totalling 54 fits
[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=1000;, score=(train=0.999, test=0.942) total time=   2.2s
[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=1000;, score=(train=0.999, test=0.932) total time=   2.2s
[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=2000;, score=(train=1.000, test=0.942) total time=   4.2s
[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=2000;, score=(train=1.000, test=0.935) total time=   4.1s
[CV 1/2] END learning_rate=0.02, max_depth=2, n_estimators=3000;, score=(train=1.000, test=0.942) total time=   6.3s
[CV 2/2] END learning_rate=0.02, max_depth=2, n_estimators=3000;, score=(train=1.000, test=0.934) total time=   6.4s
[CV 1/2] END learning_rate=0.02, max_depth=3, n_estimators=1000;, score=(train=1.000, test=0.943) total time=   2.7s
[CV 2/2] END learning_rate=0.02, max_depth=3, n_estimators=1000;, score=(train=1.000, test=0.929) total time=   3.2s
[CV

GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.25,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='auc', gamma=None, gpu_id=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_c...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, ...),
             param_grid={'learning_rate': [0.02, 0.05, 0.1],
                         'max_depth': [2, 3, 5],
                         'n_estimators': [1000, 2000, 3000]},
             return_train_score=True,
             scoring=<function auc_score at 0x0000020BD02640D0>, verbose=4)

In [55]:
df_cv_results = pd.DataFrame(model_xgboost_hp.cv_results_)
df_cv_results = df_cv_results[['rank_test_score','mean_test_score','mean_train_score',
                               'param_learning_rate', 'param_max_depth', 'param_n_estimators']]
df_cv_results.sort_values(by='rank_test_score', inplace=True)
df_cv_results

,rank_test_score,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,param_n_estimators
1,1,0.938422,1.000000,0.02,2,2000
2,2,0.937772,1.000000,0.02,2,3000
0,3,0.936851,0.998899,0.02,2,1000
9,4,0.936736,1.000000,0.05,2,1000
4,5,0.936245,1.000000,0.02,3,2000
5,6,0.936189,1.000000,0.02,3,3000
11,7,0.936110,1.000000,0.05,2,3000
12,8,0.936105,1.000000,0.05,3,1000
13,9,0.935993,1.000000,0.05,3,2000
10,10,0.935993,1.000000,0.05,2,2000
